Importing libraries

In [93]:
from langchain_openai import ChatOpenAI
import openai
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
# from pydantic import BaseModel, Field, validator
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser 
from langchain.chains import LLMChain
from PyPDF2 import PdfReader
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.output_parsers import JsonOutputParser
import os
from dotenv import load_dotenv
load_dotenv()

True

Defining the LLM

In [94]:
llm=ChatOpenAI(model="gpt-4o-mini",api_key=os.getenv('API'))

Extracting resume text

In [95]:
pdf_doc=['Resume (test1).pdf']

In [96]:
for pdf in pdf_doc:
    pdf_reader = PdfReader(pdf)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

Requirments

In [97]:
requirements='''
Deep understanding of modern machine learning techniques and algorithms.
Must demonstrate exceptional aptitude in descriptive and inferential statistics.
Extremely detail-oriented and self-motivated.
Experience with time series data.
Extensive experience working with large data.
Competent skills in programming languages such as Python, R, C/#/++.
5 years of experince in data science
'''


Creating the output purser

In [98]:
class identity(BaseModel):
    name:str=Field(description="name of the applicant")
    phone:str=Field(description="phone number of the applicant")
    email:str=Field(description="email of the applicant")
    requirement:list[str]=Field(description="list whether a requirement is fully met or not. if not, mention which criteria is not met.")
    conclusion:str=Field(description="please provide the criteria which are not met.There should be only one item in the list.")


pydantic_parser=JsonOutputParser(pydantic_object=identity)


# json_parser = SimpleJsonOutputParser()

Building the prompt and chaining prompt, LLM and output perser

In [107]:

# System Prompt
system_template='''You are an AI recruitment assistant that specializes in evaluating candidate resume.Based on the applicant's resume and job requirements given by the user,Provide the name,the phone number and the email of the applicant.The phone number needs to be in numeric. Don't add any bracket or plus sign into it.Now,Tell whether the applicant met each of the requirements. If a requirement is fully met (each criteria mentioned in the requirement), provide yes.If a requirement is not fully met, provide no. If a criteria in a requirement is not fully met, please provide the criteria which is not met.Do not miss any criteria.Return the full requirements with your evaluation.Don't just return the requirement numbers.Finally, write a short conclusion within 50 words to tell which criteria is not met by the applicant.Return the results in JSON format. Key for the requiremnts should be the whole line provided as input.'''


system_message_prompt=SystemMessagePromptTemplate.from_template(system_template)


# Human Template and chain
human_template='''
{resume},{requirements}
'''
human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)

chat_prompt=ChatPromptTemplate(messages=[system_message_prompt,human_message_prompt],input_variables=['resume','requirements'],partial_variables={'format_instructions':pydantic_parser.get_format_instructions()})

# partial_variables={'format_instructions':pydantic_parser.get_format_instructions()}

chain=chat_prompt | llm | pydantic_parser

In [108]:
result=chain.invoke({"resume":text,"requirements":requirements})

In [109]:
type(result)

dict

In [110]:
result

{'name': 'Kritika Jajoo',
 'phone': '8135399688',
 'email': 'jajoo@usf.edu',
 'requirements': {'Deep understanding of modern machine learning techniques and algorithms.': 'yes',
  'Must demonstrate exceptional aptitude in descriptive and inferential statistics.': 'no',
  'Extremely detail-oriented and self-motivated.': 'yes',
  'Experience with time series data.': 'no',
  'Extensive experience working with large data.': 'yes',
  'Competent skills in programming languages such as Python, R, C#/C++': 'no',
  '5 years of experience in data science': 'no'},
 'conclusion': 'The applicant does not meet the requirements for demonstrating exceptional aptitude in statistics, experience with time series data, competent skills in C#/C++, and lacks the required 5 years of experience in data science.'}